## Homework 3
### Cohort 1 Group7
#### Members: Simon Geller, Alex Kanstantsinau, Weixia Cheng, Mengxiao Li, Darshan Parvadiya

In [1]:
import numpy as np
import pandas as pd
from scipy.optimize import fsolve, root
import statsmodels.api as sm
import matplotlib.pyplot as plt

#### Note
We do not use the formula directly to calculate prices (because it requires that the spot curve be straight), we discount each period's coupon using the corresponding spot rate.

In [2]:
#Loading the Discount Rates
spot_df = pd.read_excel("Homework_3_Data.xlsx", skiprows=2)
spot_df.columns = ["Time", "Discount"]
spot_df['spot rate'] = ((1/spot_df['Discount'])**(1/spot_df['Time']/2)-1)*2

#Bond Class with required functionalities
class Bond:
    def __init__(self, maturity, coupon_rate):
        self.maturity = maturity
        self.coupon_rate = coupon_rate
    
    def price(self):
        time = 0
        price = 0
        while spot_df.iloc[time, 0] < self.maturity:
            price += 50 * self.coupon_rate * spot_df.iloc[time, 1]
            time += 1
        price += 100 * (1 + 0.5*self.coupon_rate) * spot_df.iloc[time, 1]
        return price
    def DV01(self):
        fdif_bond = Bond(self.maturity, self.coupon_rate + 0.0001)
        return (fdif_bond.price() - self.price()).flatten()[0]
    def Macauley(self):
        duration = 0
        time = 0
        while spot_df.iloc[time, 0] < self.maturity:
            duration += 50 * self.coupon_rate * spot_df.iloc[time, 1] * spot_df.iloc[time, 0]
            time += 1
        duration += 100 * (1+0.5*self.coupon_rate) * spot_df.iloc[time, 1] * self.maturity
        duration /= self.price()
        return duration.flatten()[0]
    def Modified(self):
        return (self.Macauley()/(1+self.coupon_rate/2)).flatten()[0]
    def convexity(self):
        convexity_value = 0
        time = 0
        while spot_df.iloc[time, 0] < self.maturity:
            t = spot_df.iloc[time, 0]
            convexity_value += 2*t * (2*t+1) * 50 * self.coupon_rate * spot_df.iloc[time, 1]
            time += 1
        t = spot_df.iloc[time, 0]
        convexity_value += 2*t * (2*t+1) * 100 * (1 + 0.5 * self.coupon_rate) * spot_df.iloc[time, 1]
        convexity_value /= (4 * self.price() * (1 + self.coupon_rate/2) ** 2)
        return convexity_value.flatten()[0]
    def second_order_shift(self, yield_change):
        return -self.Modified()*yield_change*100 + 0.5*self.convexity()*(yield_change**2)*100
    def actual_shift(self, yield_change):
        time = 0
        price = 0
        while spot_df.iloc[time, 0] < self.maturity:
            price += 50 * self.coupon_rate * (1+spot_df.iloc[time, 2]/2+yield_change/2)**(-time-1)
            time += 1
        price += 100 * (1 + 0.5*self.coupon_rate) * (1+spot_df.iloc[time, 2]/2+yield_change/2)**(-time-1)
        return price - self.price()

1. Using the spot curve in the accompanying spreadsheet, compute the par rates for
bonds with maturities of 1, 2, 3, . . ., 10 years.


In [3]:
# Method 1 use formula
spot_df['Discount cums'] = spot_df['Discount'].cumsum()
spot_df['Par Rate'] = 2*(1-1*spot_df['Discount'])/spot_df['Discount cums']

maturities = range(1, 11)
bonds = []
par_rates = np.zeros(10)
for maturity in maturities:
    rate = spot_df.loc[maturity*2-1,'Par Rate']
    bonds.append(Bond(maturity, rate))
    par_rates[maturity - 1] = rate  

pd.Series(par_rates)

0    0.030339
1    0.033504
2    0.035747
3    0.037323
4    0.038441
5    0.039265
6    0.039920
7    0.040499
8    0.041063
9    0.041644
dtype: float64

In [4]:
# Method 2 solve IRR
def par_rate(maturity):
    def equation_to_solve(coupon_rate):
        bond = Bond(maturity, coupon_rate)  
        return 100 - bond.price()
    par_rate = fsolve(equation_to_solve, x0=0.01)
    return par_rate.flatten()[0]

maturities = range(1, 11)
bonds = []
par_rates = np.zeros(10)
for maturity in maturities:
    rate = par_rate(maturity)
    bonds.append(Bond(maturity, rate))
    par_rates[maturity - 1] = rate  

pd.Series(par_rates)

0    0.030339
1    0.033504
2    0.035747
3    0.037323
4    0.038441
5    0.039265
6    0.039920
7    0.040499
8    0.041063
9    0.041644
dtype: float64

2. Compute the DV01 for each of these 10 par bonds.


In [5]:
dv01s = []
for bond in bonds:
    dv01s.append(bond.DV01())
    
pd.Series(dv01s)

0    0.009782
1    0.019227
2    0.028307
3    0.037018
4    0.045364
5    0.053356
6    0.061005
7    0.068321
8    0.075309
9    0.081976
dtype: float64

3. Compute the Macauley and modified durations for each of these 10 par bonds.


In [6]:
macauleys = []
modifieds = []
for bond in bonds:
    macauleys.append(bond.Macauley())
    modifieds.append(bond.Modified())

pd.Series(macauleys)

0    0.992521
1    1.950982
2    2.870774
3    3.751211
4    4.593355
5    5.398591
6    6.167829
7    6.901201
8    7.598125
9    8.257624
dtype: float64

In [7]:
pd.Series(modifieds)

0    0.977690
1    1.918838
2    2.820364
3    3.682489
4    4.506734
5    5.294645
6    6.047127
7    6.764227
8    7.445265
9    8.089190
dtype: float64

4. Assume that you have a $100 position in a 5-year par bond. What position in
2-year and 10-year par bonds would be needed to hedge the 5-year position against
parallel shifts in the term structure?
Demonstrate that your answer works by showing how price and reinvestment risk
offset each other if there is an immediate one-time parallel shift in the yield curve
immediately after putting on the position.


In [8]:
maturities = [2-1, 5-1, 10-1]
MCs = [macauleys[maturity] for maturity in maturities]

alpha = (MCs[1] - MCs[2])/(MCs[0] - MCs[2])
weights = np.array([alpha, 1-alpha])
print(f"The portfolio consists of {alpha:.4f} 2-year bonds and {1-alpha:.4f} 10-year bonds")

The portfolio consists of 0.5810 2-year bonds and 0.4190 10-year bonds


In [9]:
bond1,bond2,shift = [],[],[]
for i in range(-5,6,1):
    delta = i/500
    new_price_5 = bonds[4].actual_shift(delta)+bonds[4].price()
    new_price_2 = bonds[1].actual_shift(delta)+bonds[1].price()
    new_price_10 = bonds[9].actual_shift(delta)+bonds[9].price()
    prices = np.array([new_price_2,new_price_10])
    bond1.append(new_price_5)
    bond2.append(np.dot(weights,prices))
    shift.append(delta)

data = {
    'yield shift': shift,
    '5 year bond': bond1,
    '2&10 year bond': bond2,
}

df = pd.DataFrame(data)
df['delta'] = df['2&10 year bond'] - df['5 year bond']
df

,yield shift,5 year bond,2&10 year bond,delta
0,-0.010,104.627142,104.686388,5.924624e-02
1,-0.008,103.682148,103.719170,3.702211e-02
2,-0.006,102.747051,102.767219,2.016769e-02
3,-0.004,101.821735,101.830234,8.498210e-03
4,-0.002,100.906089,100.907923,1.833936e-03
5,0.000,100.000000,100.000000,4.263256e-14
6,0.002,99.103358,99.106184,2.826295e-03
7,0.004,98.216054,98.226201,1.014735e-02
8,0.006,97.337980,97.359782,2.180222e-02
9,0.008,96.469031,96.506665,3.763436e-02


5. Compute the convexities for each of these 10 par bonds.


In [10]:
convexities = [bond.convexity() for bond in bonds]
pd.Series(convexities)

0     1.440993
1     4.678763
2     9.554129
3    15.914579
4    23.620004
5    32.539475
6    42.544426
7    53.502107
8    65.271870
9    77.705700
dtype: float64

6. Use the computed dollar durations and convexities for these bonds and compute
the price change of a 100 basis point upward and downward parallel shift in the
spot curve. Compare the price changes with the actual price change obtained by
recomputing the price of the bond from the shifted spot curve.

In [11]:
shifted_prices_p = [bond.second_order_shift(0.01) for bond in bonds]
shifted_prices_n = [bond.second_order_shift(-0.01) for bond in bonds]
actual_prices_p = [bond.actual_shift(0.01) for bond in bonds]
actual_prices_n = [bond.actual_shift(-0.01) for bond in bonds]

data = {
    'actual_prices_p': actual_prices_p,
    'shifted_prices_p': shifted_prices_p,
    'actual_prices_n': actual_prices_n,
    'shifted_prices_n': shifted_prices_n,
}

df = pd.DataFrame(data)
df['delta_p']=df['shifted_prices_p']-df['actual_prices_p']
df['delta_n']=df['shifted_prices_n']-df['actual_prices_n']
df


,actual_prices_p,shifted_prices_p,actual_prices_n,shifted_prices_n,delta_p,delta_n
0,-0.970532,-0.970485,0.984943,0.984895,0.000047,-0.000048
1,-1.895672,-1.895444,1.942462,1.942232,0.000228,-0.000231
2,-2.773211,-2.772593,2.868760,2.868134,0.000618,-0.000626
3,-3.604196,-3.602916,3.763362,3.762062,0.001279,-0.001299
4,-4.390899,-4.388634,4.627142,4.624834,0.002265,-0.002308
5,-5.135566,-5.131948,5.461045,5.457343,0.003618,-0.003702
6,-5.839777,-5.834405,6.265370,6.259849,0.005372,-0.005520
7,-6.504268,-6.496716,7.039531,7.031737,0.007551,-0.007793
8,-7.129077,-7.118905,7.782164,7.771624,0.010172,-0.010540
9,-7.713898,-7.700662,8.491485,8.477719,0.013236,-0.013766
